# <center>DATABASE MODIFIER FILE</center>

### Modifying the database based on the requirements for the next 4 hours.

In [187]:
import numpy as np
import pandas as pd
from datetime import datetime, timedelta
import sys

In [188]:
pd.set_option('mode.chained_assignment', None)

In [189]:
df = pd.read_excel('bf3_data_2022_01_07.xlsx')
df.head()

,DATE_TIME,CB_FLOW,CB_PRESS,CB_TEMP,STEAM_FLOW,STEAM_TEMP,STEAM_PRESS,O2_PRESS,O2_FLOW,O2_PER,...,TOP_TEMP2,TOP_TEMP3,TOP_TEMP4,TOP_SPRAY,TOP_TEMP,TOP_PRESS_1,CO,CO2,H2,SKIN_TEMP_AVG
0,01-07-21 00:10,311727.0,3.15,129.0,4.0,213.0,3.34,3.20,7296.0,23.08,...,135.0,107.0,130.0,0.0,121.0,2.0,22.22,21.00,3.88,69.940478
1,01-07-21 00:20,315163.0,3.16,129.0,4.0,209.0,3.35,3.20,7829.0,23.08,...,143.0,109.0,128.0,0.0,125.0,1.0,22.56,21.00,3.94,71.454476
2,01-07-21 00:30,314595.0,3.16,128.0,4.0,205.0,3.35,3.21,7904.0,23.08,...,138.0,110.0,124.0,0.0,124.0,1.0,22.49,21.08,3.94,70.579462
3,01-07-21 00:40,312465.0,3.16,127.0,4.0,200.0,3.35,3.21,7919.0,23.08,...,128.0,102.0,110.0,0.0,115.0,1.0,22.36,21.13,3.99,70.179791
4,01-07-21 00:50,302981.0,3.11,126.0,4.0,194.0,3.29,3.16,7938.0,23.08,...,139.0,112.0,124.0,0.0,125.0,1.0,22.25,21.30,4.10,70.728470


In [190]:
# df = df.interpolate()

### Convert the string to DateTime Object

In [191]:
df['DATE_TIME'] = pd.to_datetime(df['DATE_TIME'], dayfirst=True)

In [192]:
df.shape

(25405, 26)

### Check for long gaps (more than 15 minutes) in timestamps and insert empty columns wherever needed.

In [193]:
actual_len = df.shape[0]

for i in range(actual_len-1):
    t1 = datetime.strptime(str(df.loc[i]['DATE_TIME']), "%Y-%m-%d %H:%M:%S")
    t2 = datetime.strptime(str(df.loc[i+1]['DATE_TIME']), "%Y-%m-%d %H:%M:%S")
    diff = t2-t1
    time_diff_min = diff.total_seconds()/60
    time_count = 10

    if(time_diff_min > 12):
        while(time_diff_min > 12):
            res = t1 + timedelta(minutes=time_count)
            time_count += 10
            time_diff_min -= 10
            df = pd.concat([df, pd.DataFrame.from_records([{'DATE_TIME': res}])], ignore_index=True)

In [194]:
df.sort_values(by=['DATE_TIME'],inplace=True ,ignore_index=True)

### Check for maximum difference in timestamps

In [195]:
max_diff = 0
time_set = set({})
for i in range(len(df)-1):
    t1 = datetime.strptime(str(df.iloc[i]['DATE_TIME']), "%Y-%m-%d %H:%M:%S")
    t2 = datetime.strptime(str(df.iloc[i+1]['DATE_TIME']), "%Y-%m-%d %H:%M:%S")
    diff = t2-t1
    time_set.add(diff.total_seconds()/60)
    max_diff = max(max_diff,diff.total_seconds()/60)
        
print(f"The max difference in time intervals : {max_diff} minutes.")

The max difference in time intervals : 12.0 minutes.


In [196]:
time_set

{8.0, 9.0, 10.0, 11.0, 12.0}

### Fill NULL values using interpolation method

In [204]:
new_df = df.copy()

In [205]:
new_df.isnull().sum()

DATE_TIME           0
CB_FLOW          3755
CB_PRESS         1122
CB_TEMP          1122
STEAM_FLOW       3755
STEAM_TEMP       1122
STEAM_PRESS      1122
O2_PRESS         1122
O2_FLOW          3755
O2_PER           1122
PCI              3755
ATM_HUMID        1122
HB_TEMP          4907
HB_PRESS         3836
TOP_PRESS        3755
TOP_TEMP1        1122
TOP_TEMP2        1122
TOP_TEMP3        1122
TOP_TEMP4        1122
TOP_SPRAY        1122
TOP_TEMP         1122
TOP_PRESS_1      3755
CO               3755
CO2              3755
H2               3755
SKIN_TEMP_AVG    1122
dtype: int64

In [206]:
new_df['DATE_TIME'] = new_df['DATE_TIME'].astype(str)

In [207]:
new_df = new_df.interpolate()

In [216]:
print(df.iloc[0]['DATE_TIME'])
print(df.iloc[24]['DATE_TIME'])

2021-07-01 00:10:00
2021-07-01 04:10:00


### Create columns for the AVG_SKIN_TEMP next 4 hours

In [217]:
SAT_1 = []
SAT_2 = []
SAT_3 = []
SAT_4 = []

for i in range(len(new_df)-24):

    SAT_1.append(new_df.iloc[i+6]['SKIN_TEMP_AVG'])
    SAT_2.append(new_df.iloc[i+12]['SKIN_TEMP_AVG'])
    SAT_3.append(new_df.iloc[i+18]['SKIN_TEMP_AVG'])
    SAT_4.append(new_df.iloc[i+24]['SKIN_TEMP_AVG'])

In [234]:
req_len = len(new_df)-24
mod_df = new_df.iloc[:req_len,:]

In [235]:
mod_df['SAT_1'] = SAT_1
mod_df['SAT_2'] = SAT_2
mod_df['SAT_3'] = SAT_3
mod_df['SAT_4'] = SAT_4

In [236]:
mod_df.head()

,DATE_TIME,CB_FLOW,CB_PRESS,CB_TEMP,STEAM_FLOW,STEAM_TEMP,STEAM_PRESS,O2_PRESS,O2_FLOW,O2_PER,...,TOP_TEMP,TOP_PRESS_1,CO,CO2,H2,SKIN_TEMP_AVG,SAT_1,SAT_2,SAT_3,SAT_4
0,2021-07-01 00:10:00,311727.0,3.15,129.0,4.0,213.0,3.34,3.20,7296.0,23.08,...,121.0,2.0,22.22,21.00,3.88,69.940478,73.583364,77.713731,82.332439,85.074844
1,2021-07-01 00:20:00,315163.0,3.16,129.0,4.0,209.0,3.35,3.20,7829.0,23.08,...,125.0,1.0,22.56,21.00,3.94,71.454476,74.666066,77.198904,82.604995,84.877672
2,2021-07-01 00:30:00,314595.0,3.16,128.0,4.0,205.0,3.35,3.21,7904.0,23.08,...,124.0,1.0,22.49,21.08,3.94,70.579462,74.837214,78.518159,84.475989,83.397999
3,2021-07-01 00:40:00,312465.0,3.16,127.0,4.0,200.0,3.35,3.21,7919.0,23.08,...,115.0,1.0,22.36,21.13,3.99,70.179791,75.008361,80.865417,84.880888,83.368013
4,2021-07-01 00:50:00,302981.0,3.11,126.0,4.0,194.0,3.29,3.16,7938.0,23.08,...,125.0,1.0,22.25,21.30,4.10,70.728470,75.799102,82.564532,84.282448,84.592822


In [237]:
mod_df.shape

(26471, 30)

In [238]:
mod_df.isna().sum()

DATE_TIME        0
CB_FLOW          0
CB_PRESS         0
CB_TEMP          0
STEAM_FLOW       0
STEAM_TEMP       0
STEAM_PRESS      0
O2_PRESS         0
O2_FLOW          0
O2_PER           0
PCI              0
ATM_HUMID        0
HB_TEMP          0
HB_PRESS         0
TOP_PRESS        0
TOP_TEMP1        0
TOP_TEMP2        0
TOP_TEMP3        0
TOP_TEMP4        0
TOP_SPRAY        0
TOP_TEMP         0
TOP_PRESS_1      0
CO               0
CO2              0
H2               0
SKIN_TEMP_AVG    0
SAT_1            0
SAT_2            0
SAT_3            0
SAT_4            0
dtype: int64

### Save the modified Dataset

In [239]:
mod_df.to_csv('modified_bf_data.csv')

# <center>---End of Notebook---</center>